In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import time
import os
import ldspec
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from collections import Counter

# autoreload
%load_ext autoreload
%autoreload 2

### Compute distance distribution

In [2]:
PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno_chimp/ukb_imp_chr@_v3_chimp'
FILE_PATH_LIST = [
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_baseline_0_100.txt',
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_baseline_0_100_ld.txt',
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_baseline_0_1000.txt',
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_baseline_0_1000_ld.txt',
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_basic.txt',
    '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list/pannot_list_gene.txt',
#     '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list_main.txt',
#     '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list.baseline_0_100.txt',
#     '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list.baseline_0_1000.txt',
#     '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_list_basic.txt',    
]
OUT_FOLDER = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot/stats'

pannot_file_list = []
for fpath in FILE_PATH_LIST:
    temp_list = pd.read_csv(fpath, header=None)[0].tolist()
    pannot_file_list += [x for x in temp_list if x not in pannot_file_list]
print('pannot_file_list', len(pannot_file_list))

for annot_file in pannot_file_list:
    annot_file = annot_file.strip()
    if annot_file.endswith((".pannot_mat.npz")) is False:
        print("    Skip: %s" % annot_file)
        continue
    annot_name = ldspec.util.get_annot_name_from_file(annot_file)
    print(annot_name)
    if os.path.exists(OUT_FOLDER+'/%s.dist.tsv' % annot_name.replace('pAN:', '')):
        continue
    
    temp_counter = Counter()
    for CHR in range(1,23):
#     for CHR in [22]:
        v_bp = ldspec.util.read_pgen(PGEN_FILE.replace('@', '%d' % CHR))['pvar']['BP'].values
        mat_G = ldspec.util.read_pannot_mat(annot_file.replace('@', '%d' % CHR))        
        for i in range(mat_G.indptr.shape[0]-1):
            temp_counter.update(v_bp[i] - v_bp[mat_G.indices[mat_G.indptr[i] : mat_G.indptr[i+1]]])

    # Hist
    temp_df = pd.DataFrame(index = np.arange(len(temp_counter)))
    temp_df['key'] = temp_counter
    temp_df['count'] = [temp_counter[x] for x in temp_counter]
    temp_df = temp_df.loc[temp_df['key']>0]
    temp_df['bin'] = pd.cut(temp_df['key'], 
                            bins=[0, 25, 5e1, 75, 1e2, 5e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6, 5e6, 1e7, 1e9])
    temp_df = temp_df.groupby('bin').agg({'count':np.sum})
    temp_df['left'] = [x.left for x in temp_df.index]
    temp_df['left'] = temp_df['left'].astype(int)
    temp_df['right'] = [x.right for x in temp_df.index]
    temp_df['right'] = temp_df['right'].astype(int)
    temp_df = temp_df[['left', 'right', 'count']]
    temp_df.to_csv(OUT_FOLDER+'/%s.dist.tsv' % annot_name.replace('pAN:', ''), sep='\t', index=True)
    print('    Finish: ', '/%s.dist.tsv' % annot_name.replace('pAN:', ''))
    
# #     break

pannot_file_list 216
pAN:H3K27ac_Hnisz_proxy_0_100_ld_full_maf_common_block
pAN:H3K27ac_Hnisz_proxy_0_100_ld_full_maf_lf_block
pAN:H3K27ac_PGC2_proxy_0_100_ld_full_maf_common_block
pAN:H3K27ac_PGC2_proxy_0_100_ld_full_maf_lf_block
pAN:H3K4me1_Trynka_proxy_0_100_ld_full_maf_common_block
pAN:H3K4me1_Trynka_proxy_0_100_ld_full_maf_lf_block
pAN:Intron_UCSC_proxy_0_100_ld_full_maf_common_block
pAN:Intron_UCSC_proxy_0_100_ld_full_maf_lf_block
pAN:Repressed_Hoffman_proxy_0_100_ld_full_maf_common_block
pAN:Repressed_Hoffman_proxy_0_100_ld_full_maf_lf_block
pAN:SuperEnhancer_Hnisz_proxy_0_100_ld_full_maf_common_block
pAN:SuperEnhancer_Hnisz_proxy_0_100_ld_full_maf_lf_block
pAN:Transcribed_Hoffman_proxy_0_100_ld_full_maf_common_block
pAN:Transcribed_Hoffman_proxy_0_100_ld_full_maf_lf_block
pAN:H3K27ac_Hnisz_proxy_0_100_ld_p0_p100_maf_common_block
pAN:H3K27ac_Hnisz_proxy_0_100_ld_p0_p100_maf_lf_block
pAN:H3K27ac_PGC2_proxy_0_100_ld_p0_p100_maf_common_block
pAN:H3K27ac_PGC2_proxy_0_100_ld_p0_p100_

    Finish:  /H3K9ac_Trynka_proxy_0_1000_ld_n100_p0_maf_lf_block.dist.tsv
pAN:Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block
    Finish:  /Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block.dist.tsv
pAN:Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block
    Finish:  /Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block.dist.tsv
pAN:Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block
    Finish:  /Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block.dist.tsv
pAN:Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block
    Finish:  /Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block.dist.tsv
pAN:Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_common_block
    Finish:  /Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_common_block.dist.tsv
pAN:Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_lf_block
    Finish:  /Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_lf_block.dist.tsv
pAN:SuperEnhancer_Hnisz_proxy_0_1000_ld_n100_p0_maf_common_block
    Finish:  /SuperEnhancer_Hnisz_proxy_0_1000_ld_n100_

In [2]:
# PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno_chimp/ukb_imp_chr@_v3_chimp'
# ANNOT_FILE = '/home/jz286/WES_analysis/LDSPEC/experiments/job.curate_data/pannot_chimp_list.txt'
# OUT_FOLDER = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/pannot/stats'

# annot_file_list = list(pd.read_csv(ANNOT_FILE, header=None)[0])
# for annot_file in annot_file_list:
#     annot_file = annot_file.strip()
#     if annot_file.endswith((".annot.gz", ".pannot_mat.npz")) is False:
#         print("    Skip: %s" % annot_file)
#         continue
#     annot_name = ldspec.util.get_annot_name_from_file(annot_file)
#     print(annot_name)
#     if os.path.exists(OUT_FOLDER+'/%s.dist.tsv' % annot_name.replace('pAN:', '')):
#         continue
    
#     temp_counter = Counter()
#     for CHR in range(1,23):
# #     for CHR in [22]:
#         v_bp = ldspec.util.read_pgen(PGEN_FILE.replace('@', '%d' % CHR))['pvar']['BP'].values
#         mat_G = ldspec.util.read_pannot_mat(annot_file.replace('@', '%d' % CHR))        
#         for i in range(mat_G.indptr.shape[0]-1):
#             temp_counter.update(v_bp[i] - v_bp[mat_G.indices[mat_G.indptr[i] : mat_G.indptr[i+1]]])

#     # Hist
#     temp_df = pd.DataFrame(index = np.arange(len(temp_counter)))
#     temp_df['key'] = temp_counter
#     temp_df['count'] = [temp_counter[x] for x in temp_counter]
#     temp_df = temp_df.loc[temp_df['key']>0]
#     temp_df['bin'] = pd.cut(temp_df['key'], 
#                             bins=[0, 25, 5e1, 75, 1e2, 5e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6, 5e6, 1e7, 1e9])
#     temp_df = temp_df.groupby('bin').agg({'count':np.sum})
#     temp_df['left'] = [x.left for x in temp_df.index]
#     temp_df['left'] = temp_df['left'].astype(int)
#     temp_df['right'] = [x.right for x in temp_df.index]
#     temp_df['right'] = temp_df['right'].astype(int)
#     temp_df = temp_df[['left', 'right', 'count']]
#     temp_df.to_csv(OUT_FOLDER+'/%s.dist.tsv' % annot_name.replace('pAN:', ''), sep='\t', index=True)
#     print('    Finish: ', '/%s.dist.tsv' % annot_name.replace('pAN:', ''))
    
# #     break

pAN:proxy_0_100_common_common
pAN:proxy_0_100_common_lf
pAN:proxy_0_100_lf_lf
pAN:proxy_0_1000_common_common
pAN:proxy_0_1000_common_lf
pAN:proxy_0_1000_lf_lf
pAN:proxy_0_10000_common_common
pAN:proxy_0_10000_common_lf
pAN:proxy_0_10000_lf_lf
pAN:ldp50p200_proxy_0_100_common_common
pAN:ldp50p200_proxy_0_100_common_lf
pAN:ldp50p200_proxy_0_100_lf_lf
pAN:ldp50p200_proxy_0_1000_common_common
pAN:ldp50p200_proxy_0_1000_common_lf
pAN:ldp50p200_proxy_0_1000_lf_lf
pAN:ldp50p200_proxy_0_10000_common_common
pAN:ldp50p200_proxy_0_10000_common_lf
pAN:ldp50p200_proxy_0_10000_lf_lf
pAN:exon_common_common
pAN:exon_common_lf
pAN:exon_lf_lf
pAN:gene_common_common
pAN:gene_common_lf
pAN:gene_lf_lf
pAN:exonic_gene_common_common
pAN:exonic_gene_common_lf
pAN:exonic_gene_lf_lf
pAN:protein_domain_common_common
pAN:protein_domain_common_lf
pAN:protein_domain_lf_lf
pAN:cS2G_promoter_common_common
pAN:cS2G_promoter_common_lf
pAN:cS2G_promoter_lf_lf
